In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from prob_unet_utils import compute_annual_block_maxima, gev_return_level, gev_parametric_bootstrap
import torch
import random
import numpy as np
import climex_utils as cu
from scipy.stats import genextreme
import matplotlib.pyplot as plt
import train_prob_unet_model as tm  

In [3]:
args = tm.get_args()

In [4]:
from prob_unet_utils import get_empirical_return_periods
def compute_return_levels_for_random_pixel(
    model,
    dataset,
    device,
    years,             
    num_samples=10,    
    chosen_pixel=None,
    variable='pr'   
):
    """
    Generate daily variable at one pixel across all days and multiple ensemble draws.
    Fit GEV, compute return levels, and do param. bootstrap for confidence intervals.
    """

    if chosen_pixel is None:
        # pick random pixel within the domain
        chosen_pixel = (random.randint(0,127), random.randint(0,127))
    pix_y, pix_x = chosen_pixel

    print(f"\n[INFO] Computing return levels at pixel = ({pix_y}, {pix_x}).")
    print(f"\n[INFO] number of samples = {num_samples}")
    print(f"\n[INFO] variable = {variable}")

    model.eval()

    
    total_days = len(dataset)  
    # we'll store daily_data shape: [total_days, num_samples]
    daily_data = np.zeros((total_days, num_samples), dtype=np.float32)

    # Iterate over each day
    for day_idx in range(total_days):
        sample_dict = dataset[day_idx]   # This loads a single day from dataset
        # Prepare inputs
        inputs = sample_dict['inputs'].unsqueeze(0).to(device)   # shape [1, C, H, W]
        lrinterp = sample_dict['lrinterp'].unsqueeze(0).to(device)
        timestamps = sample_dict['timestamps'].unsqueeze(0).to(device)

        # We'll produce 'num_samples' draws from the model
        for r in range(num_samples):
            # forward pass
            with torch.no_grad():
                output = model(inputs, t=timestamps, training=False)
            
            # convert from residual to actual
            # shape [1, nvars, H, W]
            hr_pred = dataset.residual_to_hr(output.cpu(), lrinterp.cpu())

            if variable == 'pr':
                # For precipitation: use softplus and unit conversion (kg/m²/s to mm/day)
                pr_val = cu.softplus(hr_pred[:, 0])
                pr_val = cu.kgm2sTommday(pr_val)
                pixel_val = pr_val[0, pix_y, pix_x].item()
            elif variable == 'tasmax':
                # For tasmax: compute as softplus(channel 2, c=0) + channel 1, then convert from Kelvin to Celsius
                tasmax = hr_pred[:, 1] + cu.softplus(hr_pred[:, 2], c=0)
                tasmax = cu.KToC(tasmax)
                pixel_val = tasmax[0, pix_y, pix_x].item()
            elif variable == 'tasmin':
                tasmin = hr_pred[:, 1]  # No transformation needed for tasmin
                tasmin = cu.KToC(tasmin)
                pixel_val = tasmin[0, pix_y, pix_x].item()
            else:
                raise ValueError("Unsupported variable. Use 'pr', 'tasmax' or 'tasmin'.")

            daily_data[day_idx, r] = pixel_val

    # Now daily_data shape = [total_days, num_samples]. We should have the entire record.

    # Next: get block maxima. We must know how many years we have and days_per_year
    days_per_year = 365 
    n_years = len(years) 
    # check total_days == n_years * days_per_year, or adapt as needed
    print(f"Total days = {total_days}, years = {n_years}, days_per_year = {days_per_year}")
    
    block_maxima = compute_annual_block_maxima(daily_data, years, days_per_year=days_per_year)
    # block_maxima => shape (#years * num_samples,)

    # Fit GEV
    shape_hat, loc_hat, scale_hat = genextreme.fit(block_maxima)
    print(f"GEV fit => shape={shape_hat:.3f}, loc={loc_hat:.3f}, scale={scale_hat:.3f}")

    np.save(f"{args.plotdir}/pixel_{pix_y}_{pix_x}_block_maxima.npy", block_maxima)

    # We define which return periods we want
    return_periods = [2, 5, 10, 20, 50, 100, 200, 300, 500, 700, 1000]
    # Compute return levels
    rl_values = [gev_return_level(shape_hat, loc_hat, scale_hat, T) for T in return_periods]

    # Bootstrap for confidence intervals
    rl_boot = gev_parametric_bootstrap(shape_hat, loc_hat, scale_hat,
                                       sample_size=len(block_maxima),
                                       return_periods=return_periods,
                                       n_bootstrap=1000)  

    rl_ci_lower = {}
    rl_ci_upper = {}
    for T in return_periods:
        vals_T = np.array(rl_boot[T])
        rl_ci_lower[T] = np.percentile(vals_T, 2.5)
        rl_ci_upper[T] = np.percentile(vals_T, 97.5)

    print("\nReturn Levels (mm/day) with 95% CI at the chosen pixel:")
    for T, rl in zip(return_periods, rl_values):
        ci_low = rl_ci_lower[T]
        ci_high = rl_ci_upper[T]
        print(f"  {T:3d}-year RL = {rl:.2f}  [95% CI: {ci_low:.2f}, {ci_high:.2f}]")

  
    plt.figure(figsize=(8,5))
    Ts = np.array(return_periods)
    rl_means = np.array(rl_values)
    rl_low = np.array([rl_ci_lower[T] for T in return_periods])
    rl_high= np.array([rl_ci_upper[T] for T in return_periods])

    # Plot GEV fit and confidence interval
    plt.plot(Ts, rl_means, marker='o', color='blue', label='Fitted Return Level')
    plt.fill_between(Ts, rl_low, rl_high, alpha=0.2, color='blue', label='95% CI')

    # Calculate empirical return periods and sort block maxima
    sorted_maxima, empirical_T = get_empirical_return_periods(block_maxima)

    # Plot the empirical points
    plt.scatter(empirical_T, sorted_maxima, marker='x', color='red', s=30, label='Empirical Data')

    plt.xscale('log')
    if variable == 'pr':
        plt.ylabel('Precipitation (mm/day)')
        plt.title(f"Precipitation GEV Return Levels at Pixel ({pix_y},{pix_x})")
    elif variable == 'tasmax':
        plt.ylabel('Tasmax (°C)')
        plt.title(f"Tasmax GEV Return Levels at Pixel ({pix_y},{pix_x})")
    elif variable == 'tasmin':
        plt.ylabel('Tasmin (°C)')
        plt.title(f"Tasmin GEV Return Levels at Pixel ({pix_y},{pix_x})")

    plt.xlabel('Return Period (years)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()

    # Save figure or show
    plt.savefig(f"{args.plotdir}/pixel_{pix_y}_{pix_x}_return_levels.png", dpi=200)
    plt.close()

    return daily_data, block_maxima


In [5]:
args.years_test = range(1998, 2028)
dataset_test = cu.climex2torch(
    datadir=args.datadir,
    years=args.years_test,
    variables=args.variables,
    coords=args.coords,
    lowres_scale=args.lowres_scale,
    type="lrinterp_to_residuals",
    transfo=True
)

print("Test dataset length (days):", len(dataset_test))

Opening and lazy loading netCDF files
Loading dataset into memory
Converting xarray Dataset to Pytorch tensor

##########################################
############ PROCESSING DONE #############
##########################################

Test dataset length (days): 10950


In [6]:
from prob_unet import ProbabilisticUNet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create your model with the same structure:
probunet_model = ProbabilisticUNet(
    input_channels=len(args.variables),
    num_classes=len(args.variables),
    latent_dim=16,
    num_filters=[32, 64, 128, 256],
    model_channels=32,
    channel_mult=[1, 2, 4, 8],
    beta_0=0.0,
    beta_1=0.0,
    beta_2=0.0
).to(device)

In [7]:
# Load the trained weights
checkpoint_path = "./results/plots/08/01/202508:49:05/probunet_model_lat_dim_16.pth"
probunet_model.load_state_dict(
    torch.load(checkpoint_path, map_location=device)
)
probunet_model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [8]:
daily_data, block_maxima = compute_return_levels_for_random_pixel(
    model=probunet_model,
    dataset=dataset_test,
    device=device,
    years=args.years_test,      
    num_samples=30,        
    chosen_pixel=(56, 40),
    variable= "pr"    
)


[INFO] Computing return levels at pixel = (56, 40).

[INFO] number of samples = 30

[INFO] variable = pr
Computing statistics for standardization
Total days = 10950, years = 30, days_per_year = 365
GEV fit => shape=-0.007, loc=41.307, scale=10.524

Return Levels (mm/day) with 95% CI at the chosen pixel:
    2-year RL = 45.17  [95% CI: 44.38, 46.02]
    5-year RL = 57.18  [95% CI: 55.98, 58.39]
   10-year RL = 65.18  [95% CI: 63.44, 66.91]
   20-year RL = 72.90  [95% CI: 70.41, 75.37]
   50-year RL = 82.96  [95% CI: 79.12, 86.75]
  100-year RL = 90.53  [95% CI: 85.56, 95.81]
  200-year RL = 98.12  [95% CI: 91.63, 105.33]
  300-year RL = 102.57  [95% CI: 95.18, 110.92]
  500-year RL = 108.19  [95% CI: 99.58, 118.21]
  700-year RL = 111.90  [95% CI: 102.39, 123.04]
  1000-year RL = 115.85  [95% CI: 105.25, 128.24]


In [ ]:
daily_data_tasmax, block_maxima_tasmax = compute_return_levels_for_random_pixel(
    model=probunet_model,
    dataset=dataset_test,
    device=device,
    years=args.years_test,      
    num_samples=10,        
    chosen_pixel=(56, 40),
    variable= "tasmax"    
)


[INFO] Computing return levels at pixel = (56, 40).

[INFO] number of samples = 10

[INFO] variable = tasmax
Computing statistics for standardization
Total days = 10950, years = 30, days_per_year = 365
GEV fit => shape=0.156, loc=33.022, scale=1.973

Return Levels (mm/day) with 95% CI at the chosen pixel:
    2-year RL = 33.72  [95% CI: 33.47, 33.98]
    5-year RL = 35.66  [95% CI: 35.34, 35.96]
   10-year RL = 36.77  [95% CI: 36.38, 37.12]
   20-year RL = 37.71  [95% CI: 37.21, 38.16]
   50-year RL = 38.78  [95% CI: 38.10, 39.40]
  100-year RL = 39.49  [95% CI: 38.67, 40.27]
  200-year RL = 40.13  [95% CI: 39.14, 41.09]
  300-year RL = 40.47  [95% CI: 39.38, 41.55]
  500-year RL = 40.86  [95% CI: 39.65, 42.11]
  700-year RL = 41.11  [95% CI: 39.82, 42.46]
  1000-year RL = 41.35  [95% CI: 39.97, 42.81]


In [ ]:
daily_data_tasmin, block_maxima_tasmin = compute_return_levels_for_random_pixel(
    model=probunet_model,
    dataset=dataset_test,
    device=device,
    years=args.years_test,      
    num_samples=10,        
    chosen_pixel=(56, 40),
    variable= "tasmin"    
)


[INFO] Computing return levels at pixel = (56, 40).

[INFO] number of samples = 10

[INFO] variable = tasmin


KeyboardInterrupt: 

# Observed vs. Model Return Levels Analysis

The plot comparing observed vs. model precipitation return levels reveals several important insights:

## Why are there fewer blue crosses (observed empirical) than red crosses (model empirical)?

This difference is due to sample size:
- **Observed data**: We have 30 years (1998-2028) with 1 observation per year = 30 empirical points
- **Model data**: We have 30 years with 30 samples per year = 900 empirical points (30×30)

This 30x difference explains the density difference in empirical markers.

## Interpretation of the Results:

1. **Model Underestimation**: The model GEV curve (red) falls significantly below the observed GEV curve (blue), indicating that our model systematically underestimates precipitation extremes.

2. **Statistical Significance**: The model curve falls outside the 95% confidence interval (blue shaded area) of observed GEV for most return periods, confirming this bias is statistically significant.

3. **Distribution Behavior**: 
   - **Observed data**: The empirical blue crosses align well with the fitted GEV curve
   - **Model data**: The empirical red crosses show a plateau around 75 mm/day, while the fitted model GEV continues rising - suggesting the model has an artificial ceiling for extreme precipitation

4. **Return Period Sensitivity**: The underestimation grows more severe for longer return periods, indicating our model particularly struggles with very extreme events.

5. **Physical Limitations**: The plateau in model empirical points suggests our probabilistic U-Net may have inherent limitations in generating sufficiently extreme precipitation events.

This analysis confirms our professor's approach was valuable - comparing against observed data reveals systematic biases that weren't apparent when only analyzing the model's internal consistency.

In [ ]:
# Calculate and plot the ratio between model and observed return levels
# to quantify the magnitude of the bias

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import genextreme

# First, load or calculate the necessary data if not already available
obs_block_maxima = np.load(f"{args.plotdir}/observed_pixel_56_40_block_maxima_pr.npy")
model_block_maxima = np.load(f"{args.plotdir}/pixel_56_40_block_maxima.npy")

# Fit GEV to both datasets
obs_shape, obs_loc, obs_scale = genextreme.fit(obs_block_maxima)
model_shape, model_loc, model_scale = genextreme.fit(model_block_maxima)

print(f"Observed GEV parameters: shape={obs_shape:.3f}, loc={obs_loc:.3f}, scale={obs_scale:.3f}")
print(f"Model GEV parameters: shape={model_shape:.3f}, loc={model_loc:.3f}, scale={model_scale:.3f}")

# Define return periods
return_periods = [2, 5, 10, 20, 50, 100, 200, 500, 1000]

# Calculate return levels
obs_rl = [gev_return_level(obs_shape, obs_loc, obs_scale, T) for T in return_periods]
model_rl = [gev_return_level(model_shape, model_loc, model_scale, T) for T in return_periods]

# Calculate ratios
ratios = np.array(model_rl) / np.array(obs_rl)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(return_periods, ratios, 'o-', color='purple', linewidth=2)
plt.axhline(y=1.0, color='k', linestyle='--', alpha=0.5)  # Reference line for no bias
plt.xscale('log')
plt.xlabel('Return Period (years)')
plt.ylabel('Model/Observed Return Level Ratio')
plt.title('Relative Bias in Precipitation Return Levels')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f"{args.plotdir}/return_level_bias_ratio.png", dpi=300)
plt.show()